In [1]:
using Pkg
Pkg.activate(".")
using SymPy
using LinearAlgebra

  Activating project at `c:\Users\tim\Documents\2025FallExamples\10-08`


# Afterclass wrap up
I'll redo the block-pendulum example that we analytically did in class today (8-Oct-2025)

In [11]:
g, l, m1, m2, k, c, t = symbols("g l m_1 m_2 k c t", real=true, positive=true)
f1, f2 = symbols("f_1 f_2", real=true)
x = SymFunction("x", real=true)(t)
θ = SymFunction("theta", real=true)(t)

ihat = [1,0,0]
jhat = [0,1,0]
khat = [0,0,1]
Dot(a,b) = sum( a .* b );

In [6]:
r1 = x*ihat
r2 = r1 + l*(sin(θ)*ihat - cos(θ)*jhat);

In [16]:
V = 1//2*k*x^2 + m1*g*Dot(r1,jhat) + m2*g*Dot(r2,jhat)

                            2   
                         k*x (t)
-g*l*m_2*cos(theta(t)) + -------
                            2   

In [15]:
T = 1//2*m1*Dot( diff.(r1,t), diff.(r1,t) ) + 1//2*m2*Dot( diff.(r2,t), diff.(r2,t) ) |> simplify

              2       /                 2                                      >
    /d       \        | 2 /d           \                      d            d   >
m_1*|--(x(t))|    m_2*|l *|--(theta(t))|  + 2*l*cos(theta(t))*--(theta(t))*--( >
    \dt      /        \   \dt          /                      dt           dt  >
--------------- + ------------------------------------------------------------ >
       2                                                 2                     >

>                   2\
>         /d       \ |
> x(t)) + |--(x(t))| |
>         \dt      / /
> --------------------
>                     

In [17]:
L = T-V

                                                2       /                 2    >
                                      /d       \        | 2 /d           \     >
                           2      m_1*|--(x(t))|    m_2*|l *|--(theta(t))|  +  >
                        k*x (t)       \dt      /        \   \dt          /     >
g*l*m_2*cos(theta(t)) - ------- + --------------- + -------------------------- >
                           2             2                                     >

>                                                     2\
>                   d            d          /d       \ |
> 2*l*cos(theta(t))*--(theta(t))*--(x(t)) + |--(x(t))| |
>                   dt           dt         \dt      / /
> ------------------------------------------------------
>              2                                        

In [25]:
D = 1//2*c*diff(x,t)^2

            2
  /d       \ 
c*|--(x(t))| 
  \dt      / 
-------------
      2      

In [23]:
F1 = f1*ihat
F2 = f2*ihat

Q1 = Dot(F1, diff.(r1,x)) + Dot(F2, diff.(r2,x))

f_1 + f_2

In [24]:
Q2 = Dot(F1, diff.(r1,θ)) + Dot(F2, diff.(r2,θ))

f_2*l*cos(theta(t))

In [29]:
eom = [ diff(L,diff(q,t),t) - diff(L,q) + diff(D,diff(q,t)) |> simplify for q in [x,θ]] - [Q1,Q2]

2-element Vector{Sym{PyCall.PyObject}}:
 c*Derivative(x(t), t) - f_1 - f_2 + k*x(t) + m_1*Derivative(x(t), (t, 2)) + m_2*(-l*sin(theta(t))*Derivative(theta(t), t)^2 + l*cos(theta(t))*Derivative(theta(t), (t, 2)) + Derivative(x(t), (t, 2)))
                                                                               -f_2*l*cos(theta(t)) + l*m_2*(g*sin(theta(t)) + l*Derivative(theta(t), (t, 2)) + cos(theta(t))*Derivative(x(t), (t, 2)))

If we want to reorganize this into matrix notation (to peak at the mass matrix) we can do the following

In [31]:
M = [diff(eq,diff(q,t,t)) for eq in eom, q in [x,θ] ]

2×2 Matrix{Sym{PyCall.PyObject}}:
           m_1 + m_2  l*m_2*cos(theta(t))
 l*m_2*cos(theta(t))              l^2*m_2